# Welcome to the Lab 🥼🧪

## Single Family New Rental Supply for Top SFR Markets

In this notebook, we will go over new rental supply for hot SFR markets. 

We will create a chart like this: 

![Chart](../assets/sfh_rental_supply_30_day.png)

**Note** This notebook will work with any of the 70k+ markets supported by the Parcl Labs API.

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along. 

To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY` as a secret. See this [guide](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75) for more information.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-examples/blob/main/python/inspiration/single_family_new_rental_listings_supply.ipynb)

In [ ]:
# Environment setup
import os
import sys
import subprocess
from datetime import datetime

# Collab setup from one click above
if "google.colab" in sys.modules:
    from google.colab import userdata
    %pip install parcllabs plotly kaleido
    api_key = userdata.get('PARCL_LABS_API_KEY')
else:
    api_key = os.getenv('PARCL_LABS_API_KEY')

In [ ]:
import parcllabs
import pandas as pd
import plotly.express as px
from parcllabs import ParclLabsClient

print(f"Parcl Labs Version: {parcllabs.__version__}")

In [ ]:
# initialize client
client = ParclLabsClient(api_key=api_key)

In [ ]:
# we want to isolate the top SFR markets, some of which include: 
# Atlanta, Tampa, Dallas phoenix Houston and charlotte. We previously looked these 
# metropolitan statistical areas up to get their parcl_id's

# If you wanted to look these up yourself: 

# results = client.search_markets.retrieve(
#     query='Charlotte',
#     location_type='CBSA',
#     as_dataframe=True
# )


sfr_markets = {
    'Atlanta': 2887280,
    'Tampa': 2900417,
    'Dallas': 2899734,
    'Phoenix': 2900245,
    'Houston': 2899967,
    'Charlotte': 2899841
}

sfr_names = {v: k for k, v in sfr_markets.items()}

In [ ]:
# now lets get the investor share of listings for multiple markets

rentals = client.rental_market_metrics_new_listings_for_rent_rolling_counts.retrieve_many(
    parcl_ids=sfr_markets.values(),
    params={
        'limit': 300
    },
    start_date='2023-03-01',
    as_dataframe=True,
    property_type='SINGLE_FAMILY',
)

rentals.head()

In [ ]:
# retrieve name
rentals['name'] = rentals['parcl_id'].apply(lambda x: sfr_names[x])

rentals.head()

In [ ]:
labs_logo_lookup = {
    'blue': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api.png',
    'white': 'https://parcllabs-assets.s3.amazonaws.com/powered-by-parcllabs-api-logo-white+(1).svg'
}

# set charting constants
labs_logo_dict = dict(
        source=labs_logo_lookup['white'],
        xref="paper",
        yref="paper",
        x=0.5,  # Centering the logo below the title
        y=1.02,  # Adjust this value to position the logo just below the title
        sizex=0.15, 
        sizey=0.15,
        xanchor="center",
        yanchor="bottom"
)

In [ ]:
# create chart, overlay the logo and format our viz
fig = px.line(
    rentals,
    x='date',
    y='rolling_30_day',
    color='name',  # This creates separate lines for each period
    title=f'New Single Family Home Rental Supply (Rolling 30 Day)',
    labels={'rolling_30_day': 'Rolling 30 Day SFH Rental Supply'},
    line_shape='linear'  # 'spline' for smooth lines, if preferred
)

fig.add_layout_image(labs_logo_dict)

fig.update_layout(
    margin=dict(b=100),
    height=900,
    width=1600,
    legend=dict(
        x=0.01,
        y=0.99,
        traceorder="normal",
        xanchor='left',
        yanchor='top',
        title='Market'
    ),
    title={
        'y':0.97,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    plot_bgcolor='#080D16',
    paper_bgcolor='#080D16',
    font=dict(color='#FFFFFF'),
    xaxis_title=None
)

# fig.update_yaxes(tickformat=".0%")

color_map = {
    'rolling_30_day': '#448CF2',
    'rolling_90_day': '#FFFFFF',
}

fig.update_traces(
    line=dict(width=3),
    mode='lines+markers',
    marker=dict(size=5)
)


for trace in fig.data:
    if trace.name in color_map:
        trace.line.color = color_map[trace.name]
        trace.marker.color = color_map[trace.name]

# Adding gridlines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')

# Show the plot
fig.show() # Vegas, Atlanta, San Fran, USA
fig.write_image('assets/single_family_new_rental_supply.png', width=1600, height=900)